In [13]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from category_encoders import OrdinalEncoder, OneHotEncoder

from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, plot_roc_curve
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
url = 'http://localhost:8888/edit/Dengue_features_train/dengue_labels_train.csv'
dengue_labels_train = pd.read_csv('Dengue_features_train/dengue_labels_train.csv')
dengue_features_train = pd.read_csv('Dengue_features_train/dengue_features_train.csv')
dengue_features_test = pd.read_csv('Dengue_features_train/dengue_features_test (1).csv')
submission_format = pd.read_csv('Dengue_features_train/submission_format (4).csv')

# Introduction

# Research Background

# Problem Statement(Prediction)

# Features and Observations

# Summary of the Dataset

### METHODOLOGY

In [19]:
from category_encoders import OrdinalEncoder, OneHotEncoder 
#import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np 
from sklearn.linear_model import LogisticRegression,Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.tree import export_graphviz
import graphviz
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import validation_curve
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from numpy.random import permutation
from sklearn.inspection import permutation_importance
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBClassifier
from sklearn.linear_model import LinearRegression
#from pandas_profiling import ProfileReport

In [15]:

url = 'http://localhost:8888/edit/Dengue_features_train/dengue_labels_train.csv'
dengue_labels_train = pd.read_csv('Dengue_features_train/dengue_labels_train.csv')
dengue_features_train = pd.read_csv('Dengue_features_train/dengue_features_train.csv')
dengue_features_test = pd.read_csv('Dengue_features_train/dengue_features_test (1).csv')
submission_format = pd.read_csv('Dengue_features_train/submission_format (4).csv')

In [20]:
train_join = pd.merge(dengue_labels_train,
         dengue_features_train,
         how= 'right', on=['city','year', 'weekofyear'])

### 1. WRANGLE THE DATA

In [41]:
def wrangle():  
      df = pd.merge(dengue_labels_train,
         dengue_features_train,
         how= 'right', on=['city','year', 'weekofyear'])

    # Drop constant and repeated columns
      df.drop(columns=['ndvi_ne',	'ndvi_nw','ndvi_se'], 
          inplace=True)
  
    # Drop columns with high % of NaN values
      df.dropna(axis=1, thresh=len(df)*.6, inplace=True)
      df.fillna(0, inplace=True)

      return df

In [42]:
df= wrangle()

In [24]:
X_test = wrangle(dengue_features_test)

In [ ]:
#df.isnull().sum()

In [ ]:
#dengue_labels_train.shape
#dengue_features_train.shape
#df.shape

### SPLIT THE DATA

In [25]:
# Split our feature matrix and target vector
target = 'total_cases'
y = df[target]
X = df.drop(columns=target)

In [ ]:
# Train-validation split
from sklearn.model_selection import train_test_split
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Train-test split
#from sklearn.model_selection import train_test_split
#X_test, y_test = train_test_split(dengue_features_test, test_size=0.2,random_state=42)

In [26]:
#train -valid split Train before the year 2010 and test in the year2010
mask1 = X.index <= 2009
mask2 = X.index == 2010
X_train, y_train = X.loc[mask1], y.loc[mask1]
X_val, y_val = X.loc[mask2], y.loc[mask2]

### ESTABLISH A BASELINE

In [27]:
from sklearn.metrics import mean_absolute_error
print('mean_total_cases:', y_train.mean())
y_pred = [y_train.mean()] * len(y_train)
print('Baseline MAE:', mean_absolute_error(y_train, y_pred))

mean_total_cases: 24.67513736263736
Baseline MAE: 23.003354818258664


In [28]:
from sklearn.metrics import mean_absolute_error
baseline_mae = mean_absolute_error(y_train, [y_train.mean()]*len(y_train))
print('baseline mae is', baseline_mae)

baseline mae is 23.003354818258664


### BUILD THE MODEL

In [43]:
from category_encoders import OneHotEncoder,OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

model_lr = make_pipeline(OneHotEncoder(use_cat_names=True),
                         LinearRegression())
model_lr.fit(X_train, y_train)

c:\users\israe\.virtualenvs\github-mpyjtti5\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Pipeline(steps=[('onehotencoder',
                 OneHotEncoder(cols=['city', 'week_start_date'],
                               use_cat_names=True)),
                ('linearregression', LinearRegression())])

In [44]:
coefficients = model_lr.named_steps['linearregression'].coef_
features = model_lr.named_steps['onehotencoder'].get_feature_names()
pd.Series(coefficients, features)

city_sj                       1.544472e+05
city_iq                       1.544299e+05
year                         -1.884208e+08
weekofyear                   -2.619227e+07
week_start_date_1990-04-30   -2.498907e+09
                                  ...     
station_avg_temp_c            2.068443e-01
station_diur_temp_rng_c      -1.198756e+00
station_max_temp_c            5.921605e-01
station_min_temp_c            2.329604e-01
station_precip_mm            -8.886814e-03
Length: 1070, dtype: float64

In [45]:
#Module2
#FIT GRADIENT BOOSTING MODEL
from sklearn.ensemble import GradientBoostingRegressor
oe = OrdinalEncoder()
X_train_T = oe.fit_transform(X_train)
X_val_T = oe.transform(X_val)
X_test = oe.transform(X_test)
model_gb =  make_pipeline(OrdinalEncoder(),
                         SimpleImputer(),
                         GradientBoostingRegressor(random_state=42))

model_gb.fit(X_train_T, y_train)


Pipeline(steps=[('ordinalencoder', OrdinalEncoder(cols=[], mapping=[])),
                ('simpleimputer', SimpleImputer()),
                ('gradientboostingregressor',
                 GradientBoostingRegressor(random_state=42))])

In [46]:
#Module 3 FIT XGRADIENT BOOSTING MODEL
from xgboost import XGBRegressor
oe = OrdinalEncoder()
X_train_T = oe.fit_transform(X_train)
X_val_T = oe.transform(X_val)
model_xg =  make_pipeline(OrdinalEncoder(),
                         SimpleImputer(strategy='median'),
                         XGBRegressor(random_state=42, n_jobs=10))

model_xg.fit(X_train_T, y_train)


Pipeline(steps=[('ordinalencoder', OrdinalEncoder(cols=[], mapping=[])),
                ('simpleimputer', SimpleImputer(strategy='median')),
                ('xgbregressor',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, gamma=0, gpu_id=-1,
                              importance_type='gain',
                              interaction_constraints='',
                              learning_rate=0.300000012, max_delta_step=0,
                              max_depth=6, min_child_weight=1, missing=nan,
                              monotone_constraints='()', n_estimators=100,
                              n_jobs=10, num_parallel_tree=1, random_state=42,
                              reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                              subsample=1, tree_method='exact',
                              validate_parameters=1, verbosity

In [36]:
model_rf = make_pipeline(OrdinalEncoder(),
                        SimpleImputer(strategy='median'),
                        RandomForestRegressor(random_state=42))
model_rf.fit(X_train, y_train)

Pipeline(steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['city', 'week_start_date'],
                                mapping=[{'col': 'city',
                                          'data_type': dtype('O'),
                                          'mapping': sj     1
iq     2
NaN   -2
dtype: int64},
                                         {'col': 'week_start_date',
                                          'data_type': dtype('O'),
                                          'mapping': 1990-04-30       1
1990-05-07       2
1990-05-14       3
1990-05-21       4
1990-05-28       5
              ... 
2010-06-04    1046
2010-06-11    1047
2010-06-18    1048
2010-06-25    1049
NaN             -2
Length: 1050, dtype: int64}])),
                ('simpleimputer', SimpleImputer(strategy='median')),
                ('randomforestregressor',
                 RandomForestRegressor(random_state=42))])

In [49]:
model_dt = make_pipeline(OrdinalEncoder(),
                        SimpleImputer(strategy='median'),
                        DecisionTreeRegressor(random_state=42))
model_dt.fit(X_train, y_train)

Pipeline(steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['city', 'week_start_date'],
                                mapping=[{'col': 'city',
                                          'data_type': dtype('O'),
                                          'mapping': sj     1
iq     2
NaN   -2
dtype: int64},
                                         {'col': 'week_start_date',
                                          'data_type': dtype('O'),
                                          'mapping': 1990-04-30       1
1990-05-07       2
1990-05-14       3
1990-05-21       4
1990-05-28       5
              ... 
2010-06-04    1046
2010-06-11    1047
2010-06-18    1048
2010-06-25    1049
NaN             -2
Length: 1050, dtype: int64}])),
                ('simpleimputer', SimpleImputer(strategy='median')),
                ('decisiontreeregressor',
                 DecisionTreeRegressor(random_state=42))])

In [51]:
oe = OrdinalEncoder()
oe.fit(X_train)
XT_train = oe.transform(X_train)
XT_test  = oe.transform(X_test)

In [54]:
from sklearn.linear_model import LinearRegression
model = make_pipeline(
          OrdinalEncoder(),
          SimpleImputer(),
          LinearRegression()
)
model.fit(X_train, y_train);

In [56]:
from sklearn.linear_model import Ridge
model_r =make_pipeline(
          OrdinalEncoder(),
          SimpleImputer(),
          Ridge()
)
model_r.fit(X_train, y_train)

Pipeline(steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['city', 'week_start_date'],
                                mapping=[{'col': 'city',
                                          'data_type': dtype('O'),
                                          'mapping': sj     1
iq     2
NaN   -2
dtype: int64},
                                         {'col': 'week_start_date',
                                          'data_type': dtype('O'),
                                          'mapping': 1990-04-30       1
1990-05-07       2
1990-05-14       3
1990-05-21       4
1990-05-28       5
              ... 
2010-06-04    1046
2010-06-11    1047
2010-06-18    1048
2010-06-25    1049
NaN             -2
Length: 1050, dtype: int64}])),
                ('simpleimputer', SimpleImputer()), ('ridge', Ridge())])

In [59]:
from sklearn.linear_model import LogisticRegression
model_lr = make_pipeline(
          OrdinalEncoder(),
          SimpleImputer(),
          LogisticRegression()
)
model_lr.fit(X_train, y_train);

c:\users\israe\.virtualenvs\github-mpyjtti5\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### CHECK THE METRICS

In [61]:
#Mean_Absolute_Error
from sklearn.metrics import mean_absolute_error
print('Training MAE:', mean_absolute_error(y_train, model.predict(X_train)))
print('Validity MAE:', mean_absolute_error(y_val, model.predict(X_val)))

Training MAE: 20.92588631597003


ValueError: Found array with 0 sample(s) (shape=(0, 21)) while a minimum of 1 is required.

#Root_Mean_Squared_Error

In [ ]:
from sklearn.metrics import mean_squared_error
print('Training MAE:', mean_squared_error(y_train, model.predict(X_train), squared=False))
print('Test MAE:', mean_squared_error(y_test, model.predict(X_test), squared=False))

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_train,model.predict(X_train))

### TUNE THE MODEL

### RESULTS OF THE FINIDINGS

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
feature_names = ohe.get_feature_names()
coefficients = model_lr.coef_
feature_importances = pd.Series(coefficients, index=feature_names).sort_values(key=abs)
feature_importances.tail(10).plot(kind='barh', title='Linear Coefficients')

In [ ]:
feature_names = ohe.get_feature_names()
coefficients = model_r.coef_
feature_importances = pd.Series(coefficients, index=feature_names).sort_values(key=abs)
feature_importances.tail(10).plot(kind='barh', title='Ridge Coefficients')

#Feature Importances

In [ ]:
importances = model_xg.named_steps['xgbclassifier'].feature_importances_
features = X_train.columns

feat_imp = pd.Series(importances, index=features).sort_values()
feat_imp.tail(10).plot(kind='barh')
plt.xlabel('Gini Importance')
plt.ylabel('Feature')
plt.title('Feature Importances for model_xg');